In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import LinearNDInterpolator


In [2]:
# FILEPATH: /Users/mmckay/phd_projects/analysis_routine/code/phat_catalog_interpolation.ipynb
phat_catalog_df = pd.read_csv('/Users/mmckay/phd_projects/analysis_routine/DATA/phat_f475W_f814W_table.csv')


In [3]:
phat_catalog_df.head()

,ra,dec,f475w_vega,f475w_snr,f475w_crowd,f475w_sharp,f475w_flag,f814w_vega,f814w_snr,f814w_crowd,f814w_sharp,f814w_flag
0,10.772439,41.203526,15.754,324.2,0.000,-0.023,1,14.146,562.5,0.000,-0.034,1
1,10.861762,41.237756,15.986,80.1,0.019,0.016,1,15.214,50.3,0.017,-0.050,1
2,10.763488,41.193805,15.836,253.7,0.004,0.001,1,14.699,453.0,0.002,-0.003,1
3,10.840829,41.205419,16.755,127.7,0.000,-0.048,1,15.116,435.0,0.001,-0.010,1
4,10.786556,41.173557,16.706,100.7,0.027,0.010,1,15.028,322.5,0.015,0.019,1


In [4]:
phat_catalog_df['f814w_vega_ecorr'] = phat_catalog_df['f814w_vega'] + 0.596*0.17
phat_catalog_df['f475w_vega_ecorr'] = phat_catalog_df['f475w_vega'] + 1.212*0.17
phat_catalog_df['f475w-f814w_ecorr'] = phat_catalog_df['f475w_vega_ecorr']-phat_catalog_df['f814w_vega_ecorr']
phat_catalog_df['f475w_vega-f814w_vega'] = phat_catalog_df['f475w_vega']-phat_catalog_df['f814w_vega']
phat_catalog_df = phat_catalog_df.loc[(phat_catalog_df['f814w_vega_ecorr'] <= 23.0) & (phat_catalog_df['f475w_flag'] == 1.0) & (phat_catalog_df['f814w_flag'] == 1.0)]

phat_catalog_df.to_csv('/Users/mmckay/phd_projects/analysis_routine/DATA/reduced_phat_catalog.csv')

phat_catalog_df.head()

,ra,dec,f475w_vega,f475w_snr,f475w_crowd,f475w_sharp,f475w_flag,f814w_vega,f814w_snr,f814w_crowd,f814w_sharp,f814w_flag,f814w_vega_ecorr,f475w_vega_ecorr,f475w-f814w_ecorr,f475w_vega-f814w_vega
0,10.772439,41.203526,15.754,324.2,0.000,-0.023,1,14.146,562.5,0.000,-0.034,1,14.24732,15.96004,1.71272,1.608
1,10.861762,41.237756,15.986,80.1,0.019,0.016,1,15.214,50.3,0.017,-0.050,1,15.31532,16.19204,0.87672,0.772
2,10.763488,41.193805,15.836,253.7,0.004,0.001,1,14.699,453.0,0.002,-0.003,1,14.80032,16.04204,1.24172,1.137
3,10.840829,41.205419,16.755,127.7,0.000,-0.048,1,15.116,435.0,0.001,-0.010,1,15.21732,16.96104,1.74372,1.639
4,10.786556,41.173557,16.706,100.7,0.027,0.010,1,15.028,322.5,0.015,0.019,1,15.12932,16.91204,1.78272,1.678


In [5]:
# Interpolation
RGB_merged_df = pd.read_csv('/Users/mmckay/phd_projects/analysis_routine/DATA/CMD37_csvs/RGB_isochrone_table.csv')
phat_df = pd.read_csv('/Users/mmckay/phd_projects/analysis_routine/DATA/reduced_phat_catalog.csv')


# Create a 2D array of F814W and F475W-F814W magnitudes from RGB_merged_df
cmd_arr = RGB_merged_df[['F814W_appmag', 'F475W_appmag-F814W_appmag']].values

# Mini,int_IMF,Mass,logL,logTe,logg,
# Create a LinearNDInterpolator object with 'MH' as the values
interp_MH = LinearNDInterpolator(cmd_arr, RGB_merged_df['MH'], fill_value=-99.0)
interp_Mini = LinearNDInterpolator(cmd_arr, RGB_merged_df['Mini'], fill_value=-99.0)
interp_int_IMF = LinearNDInterpolator(cmd_arr, RGB_merged_df['int_IMF'], fill_value=-99.0)
interp_Mass = LinearNDInterpolator(cmd_arr, RGB_merged_df['Mass'], fill_value=-99.0)
interp_logL = LinearNDInterpolator(cmd_arr, RGB_merged_df['logL'], fill_value=-99.0)
interp_logTe = LinearNDInterpolator(cmd_arr, RGB_merged_df['logTe'], fill_value=-99.0)
interp_logg = LinearNDInterpolator(cmd_arr, RGB_merged_df['logg'], fill_value=-99.0)



# Create a 2D array of f814w_vega and f475w-f814w magnitudes from phat_df
phat_df['f475w-f814w_ecorr'] = phat_df['f475w_vega_ecorr'] - phat_df['f814w_vega_ecorr']
# phat_df['f475w-f814w_vega'] = phat_df['f475w_vega'] - phat_df['f814w_vega']

# PHAST catalog color magnitude diagram values
filtered_arr = phat_df[['f814w_vega_ecorr', 'f475w-f814w_ecorr']].values
# filtered_arr = phat_df[['f814w_vega', 'f475w-f814w_vega']].values


# Interpolate the 'MH' values for each row in phat_df
interpolated_MH = interp_MH(filtered_arr)
interpolated_Mini = interp_Mini(filtered_arr)
interpolated_int_IMF = interp_int_IMF(filtered_arr)
interpolated_Mass = interp_Mass(filtered_arr)
interpolated_logL = interp_logL(filtered_arr)
interpolated_logTe = interp_logTe(filtered_arr)
interpolated_logg = interp_logg(filtered_arr)

# Add the interpolated 'MH' values as a new column in phat_df
phat_df['interpolated_MH'] = interpolated_MH
phat_df['interpolated_Mini'] = interpolated_Mini
phat_df['interpolated_int_IMF'] = interpolated_int_IMF
phat_df['interpolated_Mass'] = interpolated_Mass
phat_df['interpolated_logL'] = interpolated_logL
phat_df['interpolated_logTe'] = interpolated_logTe
phat_df['interpolated_logg'] = interpolated_logg

print(phat_df.shape)
print(phat_df.head(15))

phat_df = phat_df[phat_df['interpolated_MH'] != -99.0].dropna()
print(phat_df.shape)
print(phat_df.head(15))

# Save the merged dataframe as CSV
phat_df.to_csv('/Users/mmckay/phd_projects/analysis_routine/DATA/interpolated_phat_MH_catalog.csv', index=False)


(6152688, 24)
    Unnamed: 0         ra        dec  f475w_vega  f475w_snr  f475w_crowd  \
0            0  10.772439  41.203526      15.754      324.2        0.000   
1            1  10.861762  41.237756      15.986       80.1        0.019   
2            2  10.763488  41.193805      15.836      253.7        0.004   
3            3  10.840829  41.205419      16.755      127.7        0.000   
4            4  10.786556  41.173557      16.706      100.7        0.027   
5            5  10.857114  41.245740      16.289      245.9        0.001   
6            6  10.821127  41.234318      16.393      168.3        0.003   
7            7  10.703021  41.202852      19.184      526.7        0.000   
8            8  10.835287  41.209361      17.187       99.7        0.045   
9            9  10.825447  41.192824      17.721       29.3        0.000   
10          10  10.824585  41.187049      19.934      528.4        0.001   
11          11  10.835137  41.214534      20.598      305.9        0.009  

In [21]:
len(phat_df)

5889675